# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [61]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [62]:
weather_df = pd.read_csv("../output_data/cities.csv")
weather_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Port Hedland,-20.3167,118.5667,84.25,79,100,14.97,AU,1643648998
1,Portland,45.5234,-122.6762,41.90,91,75,1.01,US,1643648973
2,Rikitea,-23.1203,-134.9692,76.93,63,100,3.22,PF,1643649118
3,Port Hedland,-20.3167,118.5667,84.25,79,100,14.97,AU,1643649384
4,Portland,45.5234,-122.6762,41.90,91,75,1.01,US,1643648973
5,Rikitea,-23.1203,-134.9692,76.93,63,100,3.22,PF,1643649118
6,Albany,42.6001,-73.9662,28.74,47,41,4.00,US,1643649182
7,Busselton,-33.6500,115.3333,62.04,68,43,15.75,AU,1643649384
8,Zalegoshch',52.9008,36.8869,26.69,95,99,12.21,RU,1643649385
9,George Town,5.4112,100.3354,84.15,81,20,5.75,MY,1643649294


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [63]:
#configuring maps with api key
gmaps.configure(api_key = g_key)

#storing lats and longs in locations
locations = weather_df[["Lat", "Lng"]]

#storing humidity
humidity = weather_df["Humidity"]


In [64]:
#plotting heatmap
fig = gmaps.figure(center=(45.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

#creating heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity=200, point_radius=5)

#adding the heat layer to fig
fig.add_layer(heat_layer)

#displaying the fig
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [65]:
#filtering the cities with wind speed < 10mph, cloudines = 0, and max temp > 70 & < 80
cities_df = weather_df.loc[(weather_df["Wind Speed"] < 10) & 
                                 (weather_df["Clouds"]==0) &
                                 (weather_df["Max Temp"] > 70) &
                                 (weather_df["Max Temp"] < 80)].dropna()
cities_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
94,Salalah,17.0151,54.0924,75.29,64,0,1.14,OM,1643650028
98,Hilo,19.7297,-155.0900,76.62,60,0,4.61,US,1643656791
116,Kapaa,22.0752,-159.3190,75.18,79,0,8.05,US,1643656967
126,Cape Town,-33.9258,18.4232,79.12,77,0,1.01,ZA,1643656718
250,Salalah,17.0151,54.0924,73.49,64,0,4.61,OM,1643657034
302,Nampula,-15.1165,39.2666,76.95,78,0,9.22,MZ,1643657059
405,Sur,22.5667,59.5289,72.14,64,0,2.19,OM,1643657104
510,Beroroha,-21.6667,45.1667,78.98,44,0,5.77,MG,1643657151
560,Mocuba,-16.8392,36.9856,72.07,93,0,2.89,MZ,1643657173
663,Ixtapa,20.7000,-105.2000,78.03,50,0,9.22,MX,1643657218


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [66]:
#creating the hotel_df
hotel_df = cities_df.loc[:, ["City", "Country", "Lat", "Lng"]]

#adding a new column named Hotel Name
hotel_df["Hotel Name"] = ""
hotel_df


,City,Country,Lat,Lng,Hotel Name
94,Salalah,OM,17.0151,54.0924,
98,Hilo,US,19.7297,-155.0900,
116,Kapaa,US,22.0752,-159.3190,
126,Cape Town,ZA,-33.9258,18.4232,
250,Salalah,OM,17.0151,54.0924,
302,Nampula,MZ,-15.1165,39.2666,
405,Sur,OM,22.5667,59.5289,
510,Beroroha,MG,-21.6667,45.1667,
560,Mocuba,MZ,-16.8392,36.9856,
663,Ixtapa,MX,20.7000,-105.2000,


In [69]:
#adding the url and params
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

#pulling the city names, lats, and lngs from hotel_df
for index, row in hotel_df.iterrows():
    hotel_lat = row["Lat"]
    hotel_lng = row["Lng"]
    city_name = row["City"]
    params["location"] = f"{hotel_lat}, {hotel_lng}"
    print(f"Retrieving Reuslts for Index {index} : {city_name}")
    hotel_name = requests.get(base_url, params=params).json()
    
    results = hotel_name["results"]
    
    try: 
        print(f"Closest Hotel in {city_name} is {results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    
    except (KeyError, IndexError):
        print("Missing field/result...Skipping")
    
    print("------------------------------")
    
    
print("----------End of Search----------")
    
    

Retrieving Reuslts for Index 94 : Salalah
Closest Hotel in Salalah is Salalah Gardens Hotel
------------------------------
Retrieving Reuslts for Index 98 : Hilo
Closest Hotel in Hilo is Hilo Reeds Bay Hotel
------------------------------
Retrieving Reuslts for Index 116 : Kapaa
Closest Hotel in Kapaa is Hotel Coral Reef
------------------------------
Retrieving Reuslts for Index 126 : Cape Town
Closest Hotel in Cape Town is Protea Hotel by Marriott Cape Town Sea Point
------------------------------
Retrieving Reuslts for Index 250 : Salalah
Closest Hotel in Salalah is Salalah Gardens Hotel
------------------------------
Retrieving Reuslts for Index 302 : Nampula
Closest Hotel in Nampula is Grand Plaza Hotel
------------------------------
Retrieving Reuslts for Index 405 : Sur
Closest Hotel in Sur is Sur Plaza Hotel
------------------------------
Retrieving Reuslts for Index 510 : Beroroha
Missing field/result...Skipping
------------------------------
Retrieving Reuslts for Index 560 :

In [70]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
94,Salalah,OM,17.0151,54.0924,Salalah Gardens Hotel
98,Hilo,US,19.7297,-155.0900,Hilo Reeds Bay Hotel
116,Kapaa,US,22.0752,-159.3190,Hotel Coral Reef
126,Cape Town,ZA,-33.9258,18.4232,Protea Hotel by Marriott Cape Town Sea Point
250,Salalah,OM,17.0151,54.0924,Salalah Gardens Hotel


In [71]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [72]:
# Adding marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))